Milestone 3

Nama  : Nicholas Halasan

Batch : HCK-008

Program ini dibuat untuk melakukan automatisasi transform dan load data dari PostgreSQL ke ElasticSearch,
melakukan validasi data menggunakan Great Expectations, dan melakukan automasi
dengan membuat DAG.

# Instantiate Data Context

In [1]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

# Connect to A `Datasource`

In [38]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'nicholas-milestone3-csv'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'customer_shopping-sales'
path_to_data = 'D:\Hacktiv8\PHASE 2\m3_nicholas\p2-ftds008-hck-m3-nicholashg\data\P2M3_nicholas_clean_data.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# Create an Expectation Suite

In [39]:
# Creat an expectation suite
expectation_suite_name = 'customer-shopping-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,customer id,age,gender,item_purchased,category,purchase_amount,location,size,color,season,review_rating,subsciption_status,payment_method,shipping_type,discount_applied,promo_code,previous_purchases,preferred_payment_method,freq_of_purchases
0,1,55,Male,Blouse,Clothing,53.0,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly
1,2,19,Male,Sweater,Clothing,64.0,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly
2,3,50,Male,Jeans,Clothing,73.0,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly
3,4,21,Male,Sandals,Footwear,90.0,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly
4,5,45,Male,Blouse,Clothing,49.0,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually


## Expectations

### Expectation 1 : to be unique

In [40]:
# Expectation 1 : Column `customer_id` must be unique

validator.expect_column_values_to_be_unique('customer id')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3900,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

*INSIGHT*:
Kolom Customer ID harus unik, karena itu menggambarkan identitas pembeli dan tidak mungkin ada yang sama tiap individu

### Expectation 2 : to be between min_value and max_value

In [41]:
# Expectation 2 : Column `Review Rating` harus <= 5.0

validator.expect_column_values_to_be_between(
    column='review_rating', min_value=0.0, max_value=5.0
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3900,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**INSIGHT:**

- Dalam penjualan rating menggambarkan tingkat kepuasan customer terhadap pengalaman belanja mereka, dalam kasus ini dalam rentang 0.0 - 5.0
- Nilai Rating harus dalam range tersebut agar dapat dibandingkan rating per item dengan skala dan rentang yang sama

### Expectation 3 : to be in set

In [42]:
# Expectation 3 : Column `season` must contain one of the following  things :
#winter, fall , summer, spring

validator.expect_column_values_to_be_in_set(
    "season",
    ['Winter','Fall','Summer','Spring']
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3900,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**INSIGHT:**

Kolom Season harus diisi dengan musim yang ada di Amerika yaitu : Winter, Spring , Fall , dan Summer

### Expectation 4 : to be in type list

In [43]:
# Expectation 4 : Column `purchase_amount` must in form of int64

validator.expect_column_values_to_be_in_type_list(
    'purchase_amount', 
    ['float']
    )

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**INSIGHT:**

Pada kolom `purchase_amount` harus berisi tipe data float(desimal), karena variabel harga nilainya memungkinkan untuk tidak bulat (memiliki angka dibelakang koma)

### Expectation 5 : lengths to be between

In [44]:
# Expectation 5 : The length of the `location` column value must be less than 25

validator.expect_column_value_lengths_to_be_between(
    column='location', 
    min_value=0, 
    max_value=20
)

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3900,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**INSIGHT:**

- Setelah dicari tahu , tidak ada nama kota di Amerika yang memiliki jumlah karakter lebih dari 20
- Jika terdapat lebih dari 20 karakter, mengindikasikan ada kesalahan yang terjadi dalam memasukan nama lokasi.

### Expectation 6 : to match regex

In [45]:
# Expectation 6 : The 'age' column will match values ​​consisting of number from 0-9

validator.expect_column_values_to_match_regex(
    column='age', 
    regex=r'[1|2|3|4|5|6|7|8|9|0]+'
    )

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3900,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**INSIGHT:**

- Kolom umur harus berisi angka (dari 0-9), tidak dalam bentuk tulisan.

### Expectation 7 : to not be null

In [46]:
# Expectation 7 : Column `purchase_amount` can not contain missing values

validator.expect_column_values_to_not_be_null('purchase_amount')

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 3900,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

**INSIGHT:**
- Karena ini adalah data customer_shopping , maka yang terdapat disini adalah informasi customer yang melakukan belanja, maka tidak boleh dan tidak mungkin ada nilai yang NaN atau missing.



In [47]:
# Save into Expectation Suite

validator.save_expectation_suite(discard_failed_expectations=False)

## Checkpoint

In [48]:
# Create a checkpoint

checkpoint_1 = context.add_or_update_checkpoint(
    name = 'checkpoint_1',
    validator = validator,
)

In [49]:
# Run a checkpoint

checkpoint_result = checkpoint_1.run()

Calculating Metrics:   0%|          | 0/44 [00:00<?, ?it/s]

## Data Docs

In [50]:
# Build data docs

context.build_data_docs()

{'local_site': 'file://d:\\Hacktiv8\\PHASE 2\\m3_nicholas\\p2-ftds008-hck-m3-nicholashg\\gx\\uncommitted/data_docs/local_site/index.html'}